# Carga da Fato de Cotação

Este notebook realiza a carga da fato de cotação (fato_cotacao) a partir dos dados da tabela staging de cotação histórica.

## Imports

In [ ]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, current_timestamp


## Start Spark Session

In [3]:
# Inicializa uma SparkSession
builder = SparkSession.builder \
    .appName("Carga Fato Cotação") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado!\n")


✅ Spark 3.5.7 iniciado!



## Define Delta Table Paths

In [4]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_silver_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver"
base_gold_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold"

# Define caminhos das tabelas Delta
delta_path_cotacao_historica = f"{base_silver_path}/stg_cotacao_historica"
delta_path_dim_ativo = f"{base_gold_path}/dim_ativo_financeiro"
delta_path_fato_cotacao = f"{base_gold_path}/fato_cotacao"

## Read Source Data

In [5]:
# Lê a tabela staging de cotação histórica
df_stg_cotacao_historica = spark.read.format("delta").load(delta_path_cotacao_historica)

print(f"[SUCESSO] Leitura da tabela staging em: {delta_path_cotacao_historica}")
print(f"Total de registros: {df_stg_cotacao_historica.count()}")

df_dim_ativo = spark.read.format("delta").load(delta_path_dim_ativo)

print(f"[SUCESSO] Leitura da tabela dimensão em: {delta_path_dim_ativo}")
print(f"Total de registros: {df_dim_ativo.count()}")

[SUCESSO] Leitura da tabela staging em: D:/Projetos/Jornada_financas_pessoais/data/delta/silver/stg_cotacao_historica
Total de registros: 2396811
[SUCESSO] Leitura da tabela dimensão em: D:/Projetos/Jornada_financas_pessoais/data/delta/gold/dim_ativo_financeiro
Total de registros: 2262


## Transform Data

In [6]:
# Filtra apenas registros com tp_mercado = '10'
df_stg_cotacao_historica = df_stg_cotacao_historica.filter(col("tp_mercado") == "010")

# Join LEFT (mantém todas as cotações mesmo sem correspondência na dimensão)
df_joined = (
    df_stg_cotacao_historica.alias("stg")
    .join(
        df_dim_ativo.alias("dim"),
        col("stg.cd_negociacao") == col("dim.cd_ativo"),
        "left"  # mantém as linhas da stg mesmo se não achar na dimensão
    )
)

# Tratamento da chave surrogate faltante (usa -1)
df_fato_cotacao = (
    df_joined.select(
        when(col("dim.sk_ativo").isNull(), lit("-1")).otherwise(col("dim.sk_ativo")).alias("sk_ativo"),
        col("stg.dt_pregao"),
        col("stg.vl_abertura"),
        col("stg.vl_fechamento"),
        col("stg.vl_minimo"),
        col("stg.vl_maximo"),
        col("stg.vl_medio"),
        col("stg.vl_ultimo_negocio"),
        col("stg.qt_negocios_efetuados").alias("qt_negocios"),
        col("stg.qt_total_titulos").alias("qt_titulos"),
        col("stg.vl_total_titulos").alias("vl_volume"),
        current_timestamp().alias("ts_insercao")
    )
)

df_fato_cotacao.show(5, truncate=False)


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `stg`.`vl_fechamento` cannot be resolved. Did you mean one of the following? [`stg`.`vl_medio`, `stg`.`vl_maximo`, `stg`.`vl_minimo`, `stg`.`nm_empresa`, `stg`.`tp_mercado`].;
'Project [CASE WHEN isnull(sk_ativo#485) THEN -1 ELSE sk_ativo#485 END AS sk_ativo#911, dt_pregao#40, vl_abertura#48, 'stg.vl_fechamento, vl_minimo#50, vl_maximo#49, vl_medio#51, vl_ultimo_negocio#52, qt_negocios_efetuados#55 AS qt_negocios#912, qt_total_titulos#56 AS qt_titulos#913, vl_total_titulos#57 AS vl_volume#914, current_timestamp() AS ts_insercao#915]
+- Join LeftOuter, (cd_negociacao#42 = cd_ativo#486)
   :- SubqueryAlias stg
   :  +- Filter (tp_mercado#43 = 010)
   :     +- Relation [tp_registro#39,dt_pregao#40,cd_bdi#41,cd_negociacao#42,tp_mercado#43,nm_empresa#44,ds_especificacao_papel#45,nr_prazo_dias_mercado#46,cd_moeda_referencia#47,vl_abertura#48,vl_maximo#49,vl_minimo#50,vl_medio#51,vl_ultimo_negocio#

## Write Data

In [8]:
# Carrega a tabela Delta existente
deltaTable = DeltaTable.forPath(spark, delta_path_dim_ativo)

# Realiza o merge (upsert) na tabela Delta
(
    deltaTable.alias("target")
    .merge(
        df_dim_ativo.alias("source"),
        "target.sk_ativo = source.sk_ativo"
    )
    # Quando o registro já existe e há diferença nos campos relevantes → atualiza
    .whenMatchedUpdate(
        condition="""
            coalesce(target.ds_ativo, '') != coalesce(source.ds_ativo, '') OR 
            coalesce(target.cd_tipo_mercado, '') != coalesce(source.cd_tipo_mercado, '') OR 
            coalesce(target.ds_tipo_mercado, '') != coalesce(source.ds_tipo_mercado, '') OR 
            coalesce(target.cd_isin, '') != coalesce(source.cd_isin, '') OR
            coalesce(target.ds_tipo_ativo, '') != coalesce(source.ds_tipo_ativo, '')
        """,
        set={
            "ds_ativo": "source.ds_ativo",
            "cd_tipo_mercado": "source.cd_tipo_mercado",
            "ds_tipo_mercado": "source.ds_tipo_mercado",
            "cd_isin": "source.cd_isin",
            "ds_tipo_ativo": "source.ds_tipo_ativo",
            "ts_atualizacao": "current_timestamp()"  # Atualiza timestamp no update
        }
    )
    # Quando o registro não existe → insere novo
    .whenNotMatchedInsert(values={
        "sk_ativo": "source.sk_ativo",
        "cd_ativo": "source.cd_ativo",
        "nm_empresa": "source.nm_empresa",
        "ds_ativo": "source.ds_ativo",
        "cd_tipo_mercado": "source.cd_tipo_mercado",
        "ds_tipo_mercado": "source.ds_tipo_mercado",
        "cd_isin": "source.cd_isin",
        "ds_tipo_ativo": "source.ds_tipo_ativo",
        "ts_insercao": "current_timestamp()",  # Data/hora da inserção
        "ts_atualizacao": "null"               # Ainda não foi atualizado
    })
    .execute()
)

# 📊 Obtém métricas do último merge
last_operation = deltaTable.history(1).select("operationMetrics").collect()[0][0]
inserted = int(last_operation.get("numTargetRowsInserted", 0))
updated = int(last_operation.get("numTargetRowsUpdated", 0))

print(f"✅ Merge realizado com sucesso")
print(f"🆕 Inseridos: {inserted}")
print(f"♻️ Atualizados: {updated}")

✅ Merge realizado com sucesso
🆕 Inseridos: 2262
♻️ Atualizados: 0


## Stop Spark Session

In [9]:
# Encerra a SparkSession
spark.stop()